# 6.12.2. Función de tensión de Love en coordenadas cilíndricas

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez Marín, <daalvarez@unal.edu.co>  | Diciembre 11, 2023      | Código inicial |

## Función de tensión de Love en coordenadas cilíndricas

A continuación se definirá la función de tensión de Love para el caso de coordenadas cilíndricas a partir del vector de Galerkin.

In [1]:
from sympy import diff, Function, simplify, symbols, factor, Eq, init_printing, trigsimp, expand, latex, Matrix, zeros

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

Inicialmente definiremos algunas constantes y variables que necesitaremos para la definición de nuestro problema:

In [2]:
# constantes y variables
E, G,   = symbols("E, G")
r, t, z = symbols("r, theta, z")

# coeficiente de Poisson
nu = E/(2*G) - 1

# constante de Lamé
lamda = (nu*E)/((1 + nu)*(1 - 2*nu))

# vector de fuerzas másicas
b = zeros(3,1)

Definimos ahora los operadores básicos del cálculo vectorial en coordenadas cilíndricas:

In [3]:
# laplaciano de un campo escalar
lapl     = lambda f : (1/r)*diff(r*diff(f,r),r) + (1/(r**2))*diff(f,t,2) + diff(f,z,2)

# laplaciano vectorial en coordenadas cilíndricas
lapl_vec = lambda v : Matrix([ lapl(v[0]) - v[0]/(r**2) - (2/(r**2))*diff(v[1],t),
                               lapl(v[1]) - v[1]/(r**2) + (2/(r**2))*diff(v[0],t),
                               lapl(v[2])                                         ])

In [4]:
# gradiente de un campo escalar
grad = lambda f : Matrix([       diff(f,r),
                           (1/r)*diff(f,t),
                                 diff(f,z) ])

In [5]:
# divergencia de un campo vectorial
div = lambda v : (1/r)*diff(r*v[0],r) + (1/r)*diff(v[1],t) + diff(v[2],z)

Para definir la función de tensión de Love, haremos uso de un vector de Galerkin $\boldsymbol{p}$ de la forma:
\begin{equation*}
\boldsymbol{p} = 
   \begin{pmatrix}
            0 \\
            0 \\
            \phi(r,\theta,z)
   \end{pmatrix}
\end{equation*}

In [6]:
phi = Function("phi")(r, t, z)
p   = Matrix([ 0, 0, phi ])

y con este definiremos el campo vectorial de desplazamientos $\boldsymbol{u}$ como:
\begin{equation*}
\boldsymbol{u} = \frac{2(1 - \nu)\nabla^2 \boldsymbol{p} - \nabla(\nabla \cdot \boldsymbol{p})}{2G}.
\end{equation*}

In [7]:
# definición del campo vectorial de desplazamientos a partir del vector de Galerkin
vec_u = (2*(1 - nu)*lapl_vec(p) - grad(div(p)))/(2*G)
ur, vt, w = vec_u

Obteniendo así los desplazamientos:
\begin{align*}
u_r(r,\theta, z)      &= - \frac{1}{2 G}   \frac{\partial^2 \phi(r,\theta,z)}{\partial r\partial z}\\
v_\theta(r,\theta, z) &= - \frac{1}{2 G r} \frac{\partial^2 \phi(r,\theta,z)}{\partial \theta\partial z}\\
w(r,\theta, z)        &= \frac{1}{2 G} \left(2(1 - \nu)\nabla^2 \phi(r,\theta, z) - \frac{\partial^2 \phi(r,\theta, z)}{\partial z^2}\right)
\end{align*}   

In [8]:
imprimir(r"u_r(r,\theta,z) = ", ur)

<IPython.core.display.Math object>

In [9]:
imprimir(r"v_\theta(r,\theta,z) = ", vt)

<IPython.core.display.Math object>

In [10]:
# Se verifica la equivalencia de las expresiones
w2 = (2*(1 - nu)*lapl(phi) - diff(phi, z, 2))/(2*G)
w - w2

Al reemplazar los desplazamientos en las ecuaciones:
\begin{align*}
 \varepsilon_r &= \frac{\partial u_r}{\partial r}                                            &  \gamma_{r\theta} &= \frac{1}{r}\frac{\partial u_r}{\partial \theta} + \frac{\partial v_\theta}{\partial r} - \frac{v_\theta}{r}  \\
 \varepsilon_\theta &= \frac{u_r}{r}  + \frac{1}{r}\frac{\partial v_\theta}{\partial \theta} & \gamma_{\theta z} &= \frac{\partial v_\theta}{\partial z} + \frac{1}{r}\frac{\partial w}{\partial \theta} \label{eq:deformacion_coord_cilind} \\ 
 \varepsilon_z &= \frac{\partial w}{\partial z}                                              & \gamma_{r z} &= \frac{\partial w}{\partial r} + \frac{\partial u_r}{\partial z}.
\end{align*}

In [11]:
er  = diff(ur, r)
et  = ur/r + (1/r)*diff(vt, t)
ez  = diff(w, z)
grt = (1/r)*diff(ur, t) + diff(vt, r) - vt/r
gtz = diff(vt, z) + (1/r)*diff(w, t)
grz = diff(w, r) + diff(ur, z)

se obtienen las deformaciones longitudinales:
\begin{align*}
\varepsilon_r      &= - \frac{1}{2G}\frac{\partial^3 \phi}{\partial r^2\partial z} \\
\varepsilon_\theta &= - \frac{1}{2G}\frac{\partial}{\partial z}\left(\frac{1}{r}\frac{\partial\phi}{\partial r} + \frac{1}{r^2}\frac{\partial^2\phi}{\partial \theta^2}\right)\\
\varepsilon_z      &= \frac{1 - \nu}{G}\frac{\partial}{\partial z} \nabla^2 \phi - \frac{1}{2G}\frac{\partial^3\phi}{\partial z^3}
\end{align*}

In [12]:
imprimir(r"\varepsilon_r(r,\theta,z) = ", er)

<IPython.core.display.Math object>

In [13]:
imprimir(r"\varepsilon_\theta(r,\theta,z) = ", et)

<IPython.core.display.Math object>

In [14]:
# verificamos la equivalencia de las expresiones
ez2 = ((1 - nu)/G)*diff(lapl(phi),z) - (1/(2*G))*diff(phi, z, 3)
expand(ez - ez2)

y las angulares:
\begin{align*}
\gamma_{r\theta} &= -\frac{1}{G r}\frac{\partial^2}{\partial \theta\partial z}\left(\frac{\partial \phi}{\partial r} - \frac{\phi}{r}\right) \\
\gamma_{rz} &= \frac{1}{2 G} \frac{\partial}{\partial r}\left[2(1 - \nu) \nabla^2\phi\right] - \frac{1}{G}\frac{\partial^3 \phi}{\partial r \partial z^2}\\
\gamma_{\theta z} &= \frac{1}{2 G r} \frac{\partial}{\partial \theta}\left[2(1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2}\right] - \frac{1}{2 G r}\frac{\partial^3 \phi}{\partial \theta \partial z^2} \\
\end{align*}

In [15]:
imprimir(r"\gamma_{r\theta}(r,\theta,z) = ", grt)

<IPython.core.display.Math object>

In [16]:
# verificamos la equivalencia de las expresiones
grz2 = (1/(2*G))*diff(2*(1 - nu)*lapl(phi), r) - (1/G)*diff(phi, r, z, z)
expand(grz - grz2)

In [17]:
# verificamos la equivalencia de las expresiones
gtz2 = (1/(2*G*r))*diff(2*(1 - nu)*lapl(phi) - diff(phi, z, 2), t) - (1/(2*G*r))*diff(phi, t, z, z)
gtz - gtz2

Calculemos ahora la dilatación cúbica:

In [18]:
e = er + et + ez

para verificar que es:
\begin{equation*}
e = \frac{1}{2 G} \frac{\partial}{\partial z}\left[(1 - 2\nu) \nabla^2\phi\right]
\end{equation*}

In [19]:
e2 = (1/(2*G))*diff((1 - 2*nu)*lapl(phi),z)
expand(e - e2)

Aplicando las ecuaciones de Lamé:
\begin{align*}
   \sigma_x &= \lambda e + 2 G \varepsilon_x   &   \tau_{xy} = G\gamma_{xy} \\
   \sigma_y &= \lambda e + 2 G \varepsilon_y   &   \tau_{xz} = G\gamma_{xz} \\
   \sigma_z &= \lambda e + 2 G \varepsilon_z   &   \tau_{yz} = G\gamma_{yz}   
\end{align*}

In [20]:
sr  = lamda*e + 2*G*er
st  = lamda*e + 2*G*et
sz  = lamda*e + 2*G*ez
trt = G*grt
ttz = G*gtz
trz = G*grz

obtenemos los esfuerzos son:
\begin{align*}
\sigma_r        &= \frac{\partial}{\partial z}\left[\nu\nabla^2\phi - \frac{\partial^2 \phi}{\partial r^2}\right]\\
\sigma_\theta   &= \frac{\partial}{\partial z}\left[\nu\nabla^2\phi - \frac{1}{r}\frac{\partial\phi}{\partial r} - \frac{1}{r^2}\frac{\partial^2\phi}{\partial \theta^2}\right]\\
\sigma_z        &= \frac{\partial}{\partial z}\left[(2 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2}\right]\\
\tau_{r \theta} &= -\frac{\partial^3}{\partial r \partial \theta \partial z}\left[\frac{\phi}{r}\right]\\
\tau_{\theta z} &=  \frac{1}{r} \frac{\partial}{\partial \theta}\left[(1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2} \right]\\
\tau_{r z}      &= \frac{\partial}{\partial r}\left[(1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2} \right]
\end{align*}

In [21]:
sr2  = diff(nu*lapl(phi) - diff(phi,r,2), z)
st2  = diff(nu*lapl(phi) - (1/r)*diff(phi,r) - (1/r**2)*diff(phi, t, 2), z)
sz2  = diff((2 - nu)*lapl(phi) - diff(phi,z,2), z)
trt2 = -diff(phi/r, r, t, z)
ttz2 = (1/r)*diff((1 - nu)*lapl(phi) - diff(phi, z, 2), t)
trz2 = diff((1 - nu)*lapl(phi) - diff(phi,z,2), r)

In [22]:
simplify(sr - sr2)

In [23]:
simplify(st - st2)

In [24]:
simplify(sz - sz2)

In [25]:
simplify(trt - trt2)

In [26]:
simplify(trz - trz2)

## Función de tensión de Love en el caso axisimétrico

Observe que cuando $\phi$ es independiente de $\theta$, como en el caso de coordenadas axisimétricas, se tiene que los desplazamientos se simplifican a:

\begin{align*}
u_r(r,\theta, z)      &= - \frac{1}{2 G}   \frac{\partial^2 \phi(r,\theta,z)}{\partial r\partial z}\\
v_\theta(r,\theta, z) &= 0\\
w(r,\theta, z)        &= \frac{1}{2 G} \left(2(1 - \nu)\nabla^2 \phi(r,\theta, z) - \frac{\partial^2 \phi(r,\theta, z)}{\partial z^2}\right)
\end{align*}   

y los esfuerzos a:
\begin{align*}
\sigma_r        &= \frac{\partial}{\partial z}\left[\nu\nabla^2\phi - \frac{\partial^2 \phi}{\partial r^2}\right]\\
\sigma_\theta   &= \frac{\partial}{\partial z}\left[\nu\nabla^2\phi - \frac{1}{r}\frac{\partial\phi}{\partial r}\right]\\
\sigma_z        &= \frac{\partial}{\partial z}\left[(2 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2}\right]\\
\tau_{r \theta} &= 0 \\
\tau_{\theta z} &= 0 \\
\tau_{r z}      &= \frac{\partial}{\partial r}\left[(1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2} \right]
\end{align*}